In [5]:
!pip install mlflow keras-tuner optuna dvc -q
!pip install dvc[s3]
!apt install git -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [19]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.keras
import os
import subprocess
import optuna
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

In [ ]:
MLFLOW_TRACKING_URI = ""
EXPERIMENT_NAME = "AQI hyperparameter & model testing (Manual)"
WINDOW_SIZE = 24
PREDICT_HORIZON = 72  # Predict next 3 days = 72 hours

In [8]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/home/umair/mlruns/3', creation_time=1752022350617, experiment_id='3', last_update_time=1752022350617, lifecycle_stage='active', name='AQI hyperparameter & model testing (Manual)', tags={}>

In [9]:
!git init
!dvc init
!dvc remote add -d myremote s3://s3-bucket-umairrr

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                  

In [10]:
!git clone https://github.com/uma1r111/10pearls-AQI-Project-
%cd 10pearls-AQI-Project-

Cloning into '10pearls-AQI-Project-'...
remote: Enumerating objects: 404, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 404 (delta 80), reused 145 (delta 49), pack-reused 224 (from 1)
Receiving objects: 100% (404/404), 5.68 MiB | 26.81 MiB/s, done.
Resolving deltas: 100% (188/188), done.
/content/10pearls-AQI-Project-


In [ ]:
# Load and preprocess data
print("\nPulling latest feature_selection.csv from DVC remote (S3)...")
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = ""
subprocess.run(["dvc", "pull"], check=True)


Pulling latest feature_selection.csv from DVC remote (S3)...


CompletedProcess(args=['dvc', 'pull'], returncode=0)

In [12]:
for root, dirs, files in os.walk(".", topdown=True):
    for name in files:
        print(os.path.join(root, name))

./README.md
./.dvcignore
./requirements.txt
./.gitignore
./full_preprocessed_aqi_weather_data_with_all_features.csv
./karachi_weather_apr1_to_current.csv
./fetch_daily_data.py
./feature_selection.csv.dvc
./feature_selection.csv
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.ipynb
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.txt
./.dvc/config
./.dvc/.gitignore
./.dvc/cache/files/md5/a3/42be088d6bbebcb8002a166df6f909
./.dvc/tmp/btime
./.dvc/tmp/lock
./Data Preprocessing/Data_PreProcessing.ipynb
./Data Preprocessing/data_quality_check.py
./Data Preprocessing/run_preprocessing.py
./Data Collection/karachi_weather_jun2025.ipynb
./Data Collection/pollutants info Apr - Jun.ipynb
./Data Collection/karachi_weather_apr1_jun14.ipynb
./.github/workflows/feature_engineering.yml
./.github/workflows/feature_selection.yml
./.github/workflows/update_data.yml
./Feature Selection/initial_feature_extraction.ipynb
./Feature Selection/feature_selection.py


In [13]:
df = pd.read_csv("feature_selection.csv")

In [14]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df[(df["datetime"] >= "2025-04-01") & (df["datetime"] <= "2025-07-07")]
df = df.sort_values("datetime")

features = df.drop(columns=["datetime"])
num_features = features.shape[1]  # total features including AQI

In [15]:
print(df["datetime"].head())
print("Filtered Rows:", len(df))

0   2025-04-01 00:00:00
1   2025-04-01 01:00:00
2   2025-04-01 02:00:00
3   2025-04-01 03:00:00
4   2025-04-01 04:00:00
Name: datetime, dtype: datetime64[ns]
Filtered Rows: 2329


# **Sequence Building for Walk forward Validation**

In [16]:
def create_sequences(X, window_size, horizon):
    Xs, ys = [], []
    for i in range(len(X) - window_size - horizon):
        Xs.append(X[i:(i + window_size)].values)
        ys.append(X[(i + window_size):(i + window_size + horizon)].values)
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(features, WINDOW_SIZE, PREDICT_HORIZON)
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

In [17]:
print("Total Samples:", len(features))
print("X_seq shape:", X_seq.shape)
print("y_seq shape:", y_seq.shape)

Total Samples: 2329
X_seq shape: (2233, 24, 13)
y_seq shape: (2233, 72, 13)


In [18]:
y_train_flat = y_train.reshape(y_train.shape[0], -1)
y_val_flat = y_val.reshape(y_val.shape[0], -1)

# **Optuna** Hyperparameter Tuning

In [20]:
def optuna_objective(trial):
    n_units = trial.suggest_int("n_units", 32, 128)
    n_dense_units = trial.suggest_int("n_dense_units", 32, 128)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    recurrent_dropout = trial.suggest_float("recurrent_dropout", 0.0, 0.3)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    epochs = trial.suggest_int("epochs", 30, 50)
    n_gru_layers = trial.suggest_int("n_gru_layers", 1, 2)

    model = Sequential()
    model.add(GRU(n_units, return_sequences=(n_gru_layers > 1),
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   activation=activation,
                   dropout=dropout,
                   recurrent_dropout=recurrent_dropout))

    if n_gru_layers > 1:
        model.add(GRU(n_units, activation=activation,
                       dropout=dropout,
                       recurrent_dropout=recurrent_dropout))

    model.add(Dense(n_dense_units, activation=activation))
    model.add(Dense(PREDICT_HORIZON * features.shape[1]))  # output size = 72 x 13 = 936

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse")

    model.fit(X_train, y_train_flat,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_val, y_val_flat),
              callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
              verbose=0)

    preds = model.predict(X_val)
    rmse = sqrt(mean_squared_error(y_val_flat, preds))
    return rmse

# Run optimization
study = optuna.create_study(direction="minimize")
study.optimize(optuna_objective, n_trials=20)

# Log best trial to MLflow
with mlflow.start_run(run_name="GRU_Best_Hyperparameters_MultiOutput_Optuna"):
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)
    print("Best parameters:", study.best_params)


[I 2025-07-09 23:45:30,517] A new study created in memory with name: no-name-5f72c3fe-6c08-4d2e-9244-e9c5fe0d5f15
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


[I 2025-07-09 23:46:04,563] Trial 0 finished with value: 45.0917480796965 and parameters: {'n_units': 72, 'n_dense_units': 111, 'dropout': 0.38943939596685484, 'recurrent_dropout': 0.21412810610781427, 'learning_rate': 1.8318479020012675e-05, 'batch_size': 128, 'activation': 'tanh', 'epochs': 30, 'n_gru_layers': 1}. Best is trial 0 with value: 45.0917480796965.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step


[I 2025-07-09 23:48:39,683] Trial 1 finished with value: 26.45077332005498 and parameters: {'n_units': 64, 'n_dense_units': 107, 'dropout': 0.4893751927587222, 'recurrent_dropout': 0.20537688259905484, 'learning_rate': 0.00016889937197794456, 'batch_size': 32, 'activation': 'tanh', 'epochs': 34, 'n_gru_layers': 2}. Best is trial 1 with value: 26.45077332005498.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step


[I 2025-07-10 00:09:20,543] Trial 18 finished with value: 6.673974288837874 and parameters: {'n_units': 33, 'n_dense_units': 56, 'dropout': 0.10978463383860004, 'recurrent_dropout': 0.0879077110677477, 'learning_rate': 0.0031187060718343537, 'batch_size': 64, 'activation': 'relu', 'epochs': 38, 'n_gru_layers': 1}. Best is trial 18 with value: 6.673974288837874.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step


[I 2025-07-10 00:09:42,537] Trial 19 finished with value: 6.651661373864688 and parameters: {'n_units': 34, 'n_dense_units': 44, 'dropout': 0.10903765421610254, 'recurrent_dropout': 0.07197859168062842, 'learning_rate': 0.0025316396529251263, 'batch_size': 32, 'activation': 'relu', 'epochs': 36, 'n_gru_layers': 1}. Best is trial 19 with value: 6.651661373864688.


Best parameters: {'n_units': 34, 'n_dense_units': 44, 'dropout': 0.10903765421610254, 'recurrent_dropout': 0.07197859168062842, 'learning_rate': 0.0025316396529251263, 'batch_size': 32, 'activation': 'relu', 'epochs': 36, 'n_gru_layers': 1}
🏃 View run GRU_Best_Hyperparameters_MultiOutput_Optuna at: http://172.174.154.85:8000/#/experiments/3/runs/1913c46d3390480db6343d4cfaf2fd1f
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **LSTM Model Fit**

In [21]:
best_params = study.best_params
with mlflow.start_run(run_name="Final_GRU_MultiOutput_Optuna"):
    model = Sequential()
    model.add(GRU(best_params["n_units"],
                   return_sequences=(best_params["n_gru_layers"] > 1),
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   activation=best_params["activation"],
                   dropout=best_params["dropout"],
                   recurrent_dropout=best_params["recurrent_dropout"]))

    if best_params["n_gru_layers"] > 1:
        model.add(GRU(best_params["n_units"], activation=best_params["activation"],
                       dropout=best_params["dropout"],
                       recurrent_dropout=best_params["recurrent_dropout"]))

    model.add(Dense(best_params["n_dense_units"], activation=best_params["activation"]))
    model.add(Dense(PREDICT_HORIZON * features.shape[1]))

    model.compile(optimizer=Adam(learning_rate=best_params["learning_rate"]), loss="mse")

    model.fit(X_train, y_train_flat,
              epochs=best_params["epochs"],
              batch_size=best_params["batch_size"],
              validation_data=(X_val, y_val_flat),
              callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
              verbose=1)

    preds = model.predict(X_val)
    rmse = sqrt(mean_squared_error(y_val_flat, preds))
    mae = mean_absolute_error(y_val_flat, preds)

    # Log to MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("final_rmse", rmse)
    mlflow.log_metric("final_mae", mae)
    mlflow.set_tag("model_type", "GRU_MultiOutput")
    mlflow.set_tag("tuner", "Optuna")
    mlflow.keras.log_model(model, artifact_path="model")

    print("Final model logged to MLflow")
    print("Final RMSE:", rmse)
    print("Final MAE:", mae)

Epoch 1/36


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 1065.9235 - val_loss: 86.1233
Epoch 2/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 128.0464 - val_loss: 71.9243
Epoch 3/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 121.4745 - val_loss: 79.9479
Epoch 4/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 111.0056 - val_loss: 57.5200
Epoch 5/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 98.9828 - val_loss: 71.7469
Epoch 6/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 100.0355 - val_loss: 87.8371
Epoch 7/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 104.6704 - val_loss: 51.2182
Epoch 8/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 101.5198 - val_loss: 69.4451
Epoch 9/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 95.0890 - val_loss: 57.8804
Epoch 10/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 90.5410 - val_loss: 44.8435
Epoch 11/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 93.7850 - val_loss: 58.8613
Epoch 12/36
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 22

2025/07/10 00:10:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/10 00:10:25 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/07/10 00:10:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Final model logged to MLflow
Final RMSE: 6.519673071335762
Final MAE: 3.609173787339302
🏃 View run Final_GRU_MultiOutput_Optuna at: http://172.174.154.85:8000/#/experiments/3/runs/4e82be6c4ab94501a253b5d7e5abb613
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **Predictions**

In [22]:
last_sequence = features.values[-WINDOW_SIZE:].reshape(1, WINDOW_SIZE, features.shape[1])
future_preds = model.predict(last_sequence).reshape(PREDICT_HORIZON, features.shape[1])
future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
                             periods=PREDICT_HORIZON, freq='H')

future_df = pd.DataFrame(future_preds, columns=features.columns)
future_df["datetime"] = future_dates
future_df.to_csv("future_multivariate_predictions_optuna_GRU.csv", index=False)
mlflow.log_artifact("future_multivariate_predictions_optuna_GRU.csv")
print(future_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
    aqi_us_lag1  aqi_us_lag12  aqi_us_lag24      pm2_5  log_pm10  \
0     83.081558     83.042572     85.301582  26.178110  4.870759   
1     83.491051     82.946663     85.201134  25.733084  4.296182   
2     84.076653     82.848732     84.685234  26.076843  4.494528   
3     83.765121     82.809875     84.717148  24.921825  4.612479   
4     83.963219     83.181473     84.118782  25.219395  3.745947   
..          ...           ...           ...        ...       ...   
67    81.448792     81.791183     79.791161  27.448158  3.945252   
68    81.410927     81.463837     80.399155  27.535389  4.336591   
69    81.514725     82.217789     80.699150  28.377914  5.116654   
70    82.276917     82.335930     81.061127  28.502533  5.053318   
71    82.401817     81.999298     80.793015  28.305489  4.849917   

    scaled_humidity_%  scaled_temp_C_scaled_log_windspeed_kph   log_so2  \
0           -0.119610                                0.499324  2.88182

/tmp/ipython-input-22-963367391.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
